## S4_feature_conbined

In [5]:
import warnings; warnings.simplefilter("ignore")

import gc
import pandas as pd
from tqdm import tqdm

from utils import *

In [6]:
oof = pd.read_csv("./output/LGB_with_series_feature/oof.csv")
sub = pd.read_csv("./output/LGB_with_series_feature/submission.csv")

In [7]:
def pad_target(x): # ここに渡すoof, subはgroupby("customer_ID")もしてない(->customer_IDによって1~13レコード、数がバラバラ) --
    t = np.zeros(13)
    t[:-len(x)] = np.nan
    t[-len(x):] = x
    return list(t)

In [8]:
tmp1 = oof.groupby("customer_ID", sort=False)["target"].agg(lambda x: pad_target(x))
tmp2 = sub.groupby("customer_ID", sort=False)["prediction"].agg(lambda x: pad_target(x))

tmp = tmp1.append(tmp2)
tmp = pd.DataFrame(data=tmp.tolist(), columns=["target%s"%i for i in range(1, 14)])

In [9]:
df = []
for fn in ["cat", "num", "diff", "rank_num", "last3_cat", "last3_num", "last3_diff", "last6_num", "ym_rank_num"]:
    if len(df) == 0:
        df.append(pd.read_feather(f"./input/{fn}_feature.feather"))
    else:
        df.append(pd.read_feather(f"./input/{fn}_feature.feather").drop([id_name], axis=1))
    if "last" in fn:
        df[-1] = df[-1].add_prefix("_".join(fn.split("_")[:-1])+"_")
df.append(tmp)
df = pd.concat(df, axis=1)
print(df.shape)
df.to_feather("./input/all_feature.feather")

(1383534, 6389)


In [10]:
del oof, sub, df; gc.collect()

0

In [11]:
def one_hot_encoding(df, cols, is_drop=True):
    for col in cols:
        print("one hot encoding:", col)
        dummies = pd.get_dummies(pd.Series(df[col]), prefix="oneHot_%s"%col)
        df = pd.concat([df, dummies], axis=1)
    if is_drop:
        df.drop(cols, axis=1, inplace=True)
    return df

In [12]:
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]

In [13]:
df = pd.read_feather(f"./input/train_denoise.feather").append(pd.read_feather(f"./input/test_denoise.feather")).reset_index(drop=True)
df = df.drop(["S_2"], axis=1)
df = one_hot_encoding(df, cat_features, True)

for col in tqdm(df.columns):
    if col not in ["customer_ID", "S_2"]:
        df[col] /= 100
    df[col] = df[col].fillna(0)
df.to_feather("./input/nn_series.feather")

one hot encoding: B_30
one hot encoding: B_38
one hot encoding: D_114
one hot encoding: D_116
one hot encoding: D_117
one hot encoding: D_120
one hot encoding: D_126
one hot encoding: D_63
one hot encoding: D_64
one hot encoding: D_66
one hot encoding: D_68


100%|██████████| 224/224 [00:48<00:00,  4.58it/s]
